# Source Visualization in 3D Surface
The reason of using jupyter notebook is the 3D backend fails on vscode's jupyter support of .py file

In [5]:
import os
import mne
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from toolbox.estimate_source import SourceEstimator, get_stuff
from get_epochs import get_all_epochs

## Setup Freesurfer Environ Variables

In [6]:
if not '__file__' in dir():
    __file__ = os.path.join(os.path.abspath(''), '__fake__file__')
    
def set_freesurfer_environ():
    freesurfer_folder = os.path.join(os.path.dirname(__file__),
                                     '_link_freesurfer')
    mne.utils.set_config('SUBJECTS_DIR',
                         os.path.join(freesurfer_folder, 'subjects'))
    mne.viz.set_3d_backend('pyvista')


set_freesurfer_environ()

Using pyvista 3d backend.



## Source Estimation

In [7]:
# Settings
stc_folder = os.path.join('MiddleResults', 'SourceEstimation')

# Averaged multiple stc
# Get mutiple stcs
stcs = []
for subject_name in [f'MEG_S{e:02d}-morph' for e in range(1, 11)]:
    stcs.append(mne.read_source_estimate(os.path.join(stc_folder, subject_name)))
print(f'Got {len(stcs)} stcs')

# Average them
stc = stcs[0]
for j, s in enumerate(stcs):
    if j== 0:
        continue
    stc.data += s.data
stc.data /= len(stcs)
stc

Got 10 stcs


<SourceEstimate | 81924 vertices, tmin : -200.0 (ms), tmax : 1200.0000000000002 (ms), tstep : 10.0 (ms), data shape : (81924, 141), ~44.7 MB>

## Draw the stc using 3D-backend
A X11 frame will a time-sliding view will be established,
since the **pyvista** 3D backend is used.

**It will be slow, be patient**

In [9]:
stc2show = stc.copy()
stc2show = stc2show.crop(0, 1)

alldata = sorted(stc.data.ravel(), reverse=True)
n = len(alldata)
lims = [alldata[int(n * r)] for r in [0.05, 0.01, 0]]

clim = dict(kind='value', lims=lims)

kwargs = dict(hemi='both',
              clim=clim,
              title='Title',
              initial_time=0.3,
              time_unit='s',
              size=(800, 800),
              smoothing_steps=10,
              time_viewer=True,
              subject='fsaverage')

brain = stc2show.plot(**kwargs)

In [10]:
# If situation allows, run this block to close the brain in 3D plot
brain.close()